# UFO észlelések vizualizálása

Az adatokat a ```pandas``` segtségével kezeltem, a vizualizációt pedig a ```plotly``` segítségével csináltam.

In [1]:
import pandas as pd
import plotly.offline as py
from sklearn.preprocessing import MinMaxScaler
py.init_notebook_mode(connected=True)

### Az UFO észlelési adatok előkészítése
Az UFO észleléseket előzetesen letöltöttem egy állományba. A forrás: http://www.nuforc.org/webreports/

In [2]:
ufos = pd.read_json('../Big-Data-UFO1-homework/data.json')
ufos.columns = ufos.columns.str.lower()  # set all attribures to lower case
ufos.head()

,city,date,duration,posted,shape,state,summary,link
0,St. Louis,11/9/17 04:30,1 hour,11/9/17,Unknown,MO,White lights with a red pulsating strobe in a ...,137/S137177.html
1,Paola,11/9/17 01:00,2 hours,11/9/17,Circle,KS,Im sitting by my window smoking and happen to ...,137/S137178.html
2,Tontitown,11/8/17 22:20,2 minutes,11/9/17,Circle,AR,Large solid white ball.,137/S137176.html
3,Loveland,11/8/17 20:50,20 minutes,11/9/17,Changing,CO,5 glimmering stationary objects for 20 minutes...,137/S137180.html
4,Plainville,11/8/17 20:35,5 minutes,11/9/17,Formation,CT,Nov 8 at approx 8:35 pm driving west on rte 37...,137/S137181.html


A ```summary``` és a ```link``` mezőkre most nem lesz szükség.

In [3]:
ufos.drop(columns=['summary', 'link'], inplace=True)

Kérdés, hogy milyen típusúak az az attribútumok és hogy tartalmaznak-e NULL értékeket.

In [4]:
pd.DataFrame({'null': ufos.isnull().any(), 'types': ufos.dtypes})

,null,types
city,False,object
date,False,object
duration,False,object
posted,False,object
shape,False,object
state,False,object


Bár a mezők sehol sem tartalmaznak ```null```-t, mivel ```object``` típusúak, ezért még tartalmazhatnak azzal elvivalens értékeket.

Mivel a ```Date``` mező jelenleg nem dátum formátumú, ezért átalakítom arra. Nem mindenhol sikerült, ezért megnézem hogy ahol nem sikerült, ott miért nem (a ```coerce``` kapcsoló ```null```-t tesz a sikertelen parsolás helyére.)

In [5]:
ufos['date_parsed'] = pd.to_datetime(ufos['date'], errors='coerce')
ufos[ufos['date_parsed'].isnull()].head()

,city,date,duration,posted,shape,state,date_parsed
553,Olympia,md 9 10:10,2 minutes,10/19/17,,WA,NaT
827,Hellertown,,2 seconds,9/28/17,,PA,NaT
1172,Copalis,,,9/5/17,,WA,NaT
2187,Shepherdstown,,10 minutes,7/7/17,Oval,WV,NaT
2929,Atlantic City,4/ 23:22,30 minutes,5/6/17,Disk,NJ,NaT


Úgy tűnik hogy a forrásban nem szabványos módon is vannak tárolva dátumok, illetve van ahol nincs is kitöltve.

A ```City``` mező sok helyen zárójelek között tartalmaz egyéb földrajzi információkat, amelyeket eltávolítok a sorok végéről. Az így kapott neveket a ```City_cleaned``` változóba teszem el és a továbbiakban ezt használom a városok azonosítására.

In [6]:
ufos['city_cleaned'] = ufos['city'].str.replace(r'[(][A-Za-z/]*[)]$', '').str.rstrip()
ufos[ufos['city_cleaned'] != ufos['city']].head()

,city,date,duration,posted,shape,state,date_parsed,city_cleaned
13,Pueblo (Belmont),11/7/17 17:15,1 minute,11/9/17,Other,CO,2017-11-07 17:15:00,Pueblo
21,Acol (UK/England),11/6/17 19:45,Still ongoing,11/9/17,Flash,,2017-11-06 19:45:00,Acol
32,Highway 60 (Canada),11/5/17 19:38,1 second,11/9/17,Circle,MB,2017-11-05 19:38:00,Highway 60
64,Athabasca (Canada),10/27/17 17:30,4 seconds,11/9/17,Rectangle,AB,2017-10-27 17:30:00,Athabasca
69,New York City (Brooklyn),8/4/17 00:00,20 minutes,11/9/17,Formation,NY,2017-08-04 00:00:00,New York City


### Az USA város listájának előkészítése
Az USA városainak listája: https://simplemaps.com/data/us-cities

In [7]:
cities = pd.read_csv('../Big-Data-UFO1-homework/uscitiesv1.4.csv')
cities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659


Törlöm a fölösleges attribútumokat. (Az egyes attribútumok pontos jelentése a fenti linken elérhető.)

In [8]:
cities.drop(columns=['county_fips', 'source', 'zips', 'id'], inplace=True)

Kérdés, hogy milyen típusúak az az attribútumok és hogy tartalmaznak-e NULL értékeket.

In [9]:
pd.DataFrame({'null': cities.isnull().any(), 'types': cities.dtypes})

,null,types
city,False,object
city_ascii,False,object
state_id,False,object
state_name,False,object
county_name,False,object
lat,False,float64
lng,False,float64
population,True,float64
population_proper,True,float64
density,False,float64


Látszik, hogy lokációs adatok mindenhol szerepelnek, viszont a populáció nem. TODO: Azokat a városokat is törölnöm kell. Hasonlóan az 5 főnél kisebb lélekszámú településeket.

### Az adatok összefésülése az UFO észlelések helyének ellenőrzésére
A két táblát a ```City_cleaned``` és a ```State``` mező mentén szeretném összefésülni. A ```City``` mezőt már az észlelések előkészítésénél kitisztítottam, most a ```State``` mezőt nézem meg.

In [10]:
set(ufos['state'].unique()) - set(cities['state_id'].unique())

{'',
 'AB',
 'BC',
 'Ca',
 'Fl',
 'MB',
 'NB',
 'NF',
 'NS',
 'NT',
 'ON',
 'PE',
 'PQ',
 'QC',
 'SA',
 'SK',
 'YK',
 'YT'}

Ezek szerint egyrészt, az ```ufos``` táblában vannak hibás, vagy más országhoz tartozó rövidítések is. Másrészt, van ahol nem nagy betűvel van írva California állam rövidítése.

In [11]:
ufos[ufos['state'] == 'Ca'] = 'CA'
ufos[ufos['state'] == ''] = None

Összefésülöm az UFO észleléseket a városok neve és az államok rövidítései mentén a ```mdata``` (merged data) táblába.

In [12]:
mdata = pd.merge(ufos, cities, left_on=['city_cleaned', 'state'], right_on=['city', 'state_id']).drop(columns=['city_y', 'state'])
mdata.head()

,city_x,date,duration,posted,shape,date_parsed,city_cleaned,city_ascii,state_id,state_name,county_name,lat,lng,population,population_proper,density,incorporated,timezone
0,Paola,11/9/17 01:00,2 hours,11/9/17,Circle,2017-11-09 01:00:00,Paola,Paola,KS,Kansas,Miami,38.5783,-94.8626,5366.0,5568.0,441.0,True,America/Chicago
1,Paola,9/1/99 21:00,<1 minute,9/9/13,Cigar,1999-09-01 21:00:00,Paola,Paola,KS,Kansas,Miami,38.5783,-94.8626,5366.0,5568.0,441.0,True,America/Chicago
2,Paola,7/16/10 19:30,40 Minutes,7/19/10,Cigar,2010-07-16 19:30:00,Paola,Paola,KS,Kansas,Miami,38.5783,-94.8626,5366.0,5568.0,441.0,True,America/Chicago
3,Paola,2/9/06 05:45,3-5 minutes,3/11/06,Unknown,2006-02-09 05:45:00,Paola,Paola,KS,Kansas,Miami,38.5783,-94.8626,5366.0,5568.0,441.0,True,America/Chicago
4,Paola,12/15/04 09:00,2 min,1/11/05,Triangle,2004-12-15 09:00:00,Paola,Paola,KS,Kansas,Miami,38.5783,-94.8626,5366.0,5568.0,441.0,True,America/Chicago


Az előbbi lépésben előállítottam a két tábla metszetét a két-két attribútum mentén (inner join). Kérdés, hogy hány észlelés maradt így.

In [13]:
print('Az eredeti észlelés tábla mérete:', len(ufos))
print('A városokkal összefésült tábla mérete:', len(mdata))
print('Maradt %:', len(mdata)/len(ufos))

Az eredeti észlelés tábla mérete: 113319
A városokkal összefésült tábla mérete: 82544
Maradt %: 0.7284215356648047


Az eredeti halmaz mintegy negyedét elveszítettem, de a maradék legalább biztosan amerikai város, amelyekhez tartozik hely és lakosság sűrűség információ.

### Az UFO észlelések száma a népsűrűség függvényében

Feltételezem, hogy az UFO észlelések száma korrelál a népsűrűséggel, ezért normálom az észlelések számát a sűrűséggel.

Az észleléseket csoportosítom a települések szerint és meghatározom az egyes csoportok számát. Az eredményt ```DataFrame``` táblává alakítom.

In [14]:
ufos_seen = pd.DataFrame(mdata.groupby(['state_id', 'city_cleaned']).size(), columns=['seen_count'])
ufos_seen.head()

seen_count
state_id city_cleaned            
AK       Adak                   2
         Anchor Point           3
         Anchorage            130
         Angoon                 1
         Auke Bay               2

Azért hogy ki tudjam egészíteni az észlelések számát a sűrűség adatokkal, összefésülöm (újra) a ```cities``` táblával, majd pedig újraindexelem a kapott táblát, hogy továbbra is a ```state_id``` és a ```city``` legyenek a kulcsok.

In [15]:
ufos_seen = pd.merge(ufos_seen, cities, left_index=True, right_on=['state_id', 'city'])
ufos_seen.set_index(['state_id', 'city'], inplace=True)
ufos_seen[['seen_count', 'density']].head()

seen_count  density
state_id city                             
AK       Adak                   2      3.0
         Anchor Point           3     59.1
         Anchorage            130     67.0
         Angoon                 1      7.0
         Auke Bay               2    470.1

A következőekben el szeretném osztani a ```density``` változó értékét. Ezért ellenőrzöm, hogy nem 0-e, hogy ne kapjak ```Inf``` értékeket.

In [16]:
ufos_seen[ufos_seen['density'] == 0.0][['seen_count', 'density']]

seen_count  density
state_id city                                
AK       Cold Bay                  1      0.0
         Wrangell                  3      0.0
CA       Bucks Lake                1      0.0
FL       Bay Lake                  1      0.0
OR       White City                5      0.0
PA       King of Prussia           5      0.0

A talált rekordokat eldobom.

In [17]:
ufos_seen = ufos_seen[ufos_seen['density'] != 0.0]

Mivel a népesség adatok nem állnak rendelkezésre minden város esetében, ezért az UFO észlelések számát a népsűrűség változóval normálom.

In [18]:
ufos_seen['ufos_seen_by_density'] = ufos_seen['seen_count'] / ufos_seen['density']
ufos_seen[['seen_count', 'density', 'ufos_seen_by_density']].head()

seen_count  density  ufos_seen_by_density
state_id city                                                   
AK       Adak                   2      3.0              0.666667
         Anchor Point           3     59.1              0.050761
         Anchorage            130     67.0              1.940299
         Angoon                 1      7.0              0.142857
         Auke Bay               2    470.1              0.004254

Mivel az UFO-k feltehetőleg rendkívül látványos jelenségek, ott ahol tényleg láttak egyet, ott valószínűleg több ember is jelentette. Ezért először rendezem az adatokat a terület/ember-re jutó UFO észlelések száma szerint ```ufos_seen_by_density```, majd pedig megnézem hogy melyek a legnagyobb ilyen települések, amelyeket kiteszek a térképre.

In [19]:
cities_with_most_ufos = ufos_seen.sort_values(by='ufos_seen_by_density', ascending=False).iloc[:500, :]
cities_with_most_ufos[['seen_count', 'density', 'ufos_seen_by_density']].head(10)

,,seen_count,density,ufos_seen_by_density
state_id,city,,,
FL,Merritt Island,23,0.1,230.000000
AK,Nelchina,1,0.1,10.000000
FL,Lake Buena Vista,7,1.0,7.000000
CA,Caribou,1,0.2,5.000000
AK,Juneau,17,4.0,4.250000
FL,Yeehaw Junction,4,1.1,3.636364
AK,Sitka,3,1.0,3.000000
PA,Dingmans Ferry,5,1.8,2.777778
NV,Valmy,1,0.4,2.500000


A 500 legnépszerűbb UFO látogatási hely közül kiválaszom a legnagyobb városokat, amelyeket feltszek a térképre.

A térképen a pontok méretét az UFO észlelések lakosságra vett sűrűségének a logaritmusa adja, a pontok színét pedig az abszolút száma az észleléseknek. Ehhez szükség van néhány további változóra...

In [20]:
log_dens = pd.np.log(cities_with_most_ufos['ufos_seen_by_density'])
scaled_log_dens = MinMaxScaler(copy=True, feature_range=(5,15)).fit_transform(log_dens.reshape(-1, 1)).reshape(-1)
density_str = cities_with_most_ufos['ufos_seen_by_density'].round(2).astype(str)
density_info = cities_with_most_ufos['city_ascii'] + ' (Density: ' + density_str + ') '

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



Az alábbi kódrészlet nagyrészt a Plotly oldaláról van átvéve: https://plot.ly/python/scatter-plots-on-maps/

In [21]:
colorscale = [
    [0, "rgb(5, 10, 172)"],
    [0.35, "rgb(40, 60, 190)"],
    [0.5, "rgb(70, 100, 245)"],
    [0.6, "rgb(90, 120, 245)"],
    [0.7, "rgb(106, 137, 247)"],
    [1, "rgb(220, 220, 220)"]
]

data = [{
    'type': 'scattergeo',
    'locationmode': 'USA-states',
    'lon': cities_with_most_ufos['lng'],
    'lat': cities_with_most_ufos['lat'],
    'text': density_info,
    'mode': 'markers',
    'marker': {
        'size': scaled_log_dens,
        'opacity': 0.8,
        'reversescale': True,
        'autocolorscale': False,
        'symbol': 'circle',
        'line': {
            'width': 1,
            'color': 'rgba(102, 102, 102)',
        },
        'colorscale': colorscale,
        'cmin': 0,
        'color': cities_with_most_ufos['seen_count'],
        'cmax': cities_with_most_ufos['seen_count'].max(),
        'colorbar': {
            'title': 'Ufos seen',
        },
    }
}]

layout = {
    'title': 'UFO reports in the USA',
    'colorbar': True,
    'geo': {
        'scope': 'usa',
        'projection': { 
            'type': 'albers usa',
        },
        'showland': True,
        'landcolor': "rgb(250, 250, 250)",
        'subunitcolor': "rgb(217, 217, 217)",
        'countrycolor': "rgb(217, 217, 217)",
        'countrywidth': 0.5,
        'subunitwidth': 0.5,
    },
}

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='ufos' )